# <span style="color:red;"> Semáforos</span>

Ya se han visto el la teoría la diferencia entre hilos y procesos. En este notebook vamos a aprender a utilizar semáforos en hilos en lugar de procesos porque simplifica la compartición de variables.

## <span style="color:orange;">Funcionalidad</span>

Un semáforo se puede ver como una variable que tiene un valor entero:
* Puede iniciarse con un valor no negativo.
* La operación down (wait) (P según la notación Dijkstra):
    * Si semaforo > 0: disminuye el valor del semáforo
    * Si semaforo = 0: el proceso se duerme.
* La operación up (signal) (V según la notación Dijkstra) incrementa el valor del semáforo.
    * Si hay procesos durmiendo en ese semáforo despierta a alguno de ellos.
    * De lo contrario incrementa el valor del semáforo.

## <span style="color:orange;">Generalizando un poco</span>

Vamos a hacer algo un poco más genérico en el que creamos un procedimiento Launcher. En el primer parámetro se almacenan los identificadores de los hilos en forma de lista. Es una función de parámetros variables en el que se pasa un conjunto de hilos a crear e iniciar. Con el fin de iniciar todos los hilos lo más simultáneamente posible se espera a que sean creados todos antes de ser lanzados.

Así mismo vamos a empezar a trabajar con semáforos. Por simplicidad para aprender el uso de semáforos vamos a utilizar los semáforos de hilos para aprender a trabajar con ellos. En el código de la siguiente celda también se dedinen los procedimientos _up_ y _down_ con el fin de que los problemas a resolver tengan un aspecto muy similar al pseudocódigo de los problemas típicos.

In [1]:
from threading import Thread, Semaphore
from time import sleep
from random import choice,randint
from string import ascii_letters
import os


def Launcher(threads,*funcs):
    for func in funcs:
        t = Thread(target=func)
        threads.append(t)
    for thr in threads:
        thr.start()
        
def down(sem):
    sem.acquire()

def up(sem):
    sem.release()
    
def SemaphoreArray(N,value=1):
    array = []
    for i in range(N):
        array.append(Semaphore(value))
    return array

Vamos a hacer una pequeña prueba de estos procedimientos. Creamos cuatro procedimientos cada uno controlado por un semáforo inicializado a 0 de tal forma que los hilos se duermen al hacer el _down_ al principio del hilo. En el procedimiento principal haremos un _up_ de cada uno de estos semáforos.

Prueba varias veces el código anterior. ¿Por qué no aparecen los mensajes en el mismo orden en el que se hacen los _up_?

## <span style="color:orange;">Problemas típicos con semáforos</span>
### <span style="color:yellow;">Problema del Productor - consumidor</span>

#### <span style="color:cyan;">Enunciado del productor-consumidor</span>

Uno o varios procesos producen un bien que guardan en un almacén y un o varios procesos consumen ese bien del almacén. El almacé tendrá un tamaño limitado.

#### <span style="color:cyan;">Implementación del productor-consumidor</span>

Vamos a crear un productor y un consumidor con un almacén de tamaño limitado (N). Para que estos hilos no sean infinitos se sustituye el bucle infinito por un bucle de tamaño limitado. Así mismo, para poder controlar cuándo se ejecutan los dos procesos se añade un semáforo controlador para el productor y otro para el consumidor.

#### <span style="color:cyan;">Cuestiones</span>

1. Intenta explicar el resultado.
2. ¿Qué sucede si se quita el último sleep? Pruébalo y explica qué sucede.
3. ¿Qué sucede si se cambia el orden relativo del mutex respecto a los semáforos de vacíos y llenos. Pruébalo.
4. ¿El código es válido para múltiples productores y consumidores? Haz las pruebas necesarias.

## <span style="color:orange;">Problema del baile de salón</span>
### <span style="color:yellow;">Enunciado del problema del baile de salón</span>
En el hotel Hastor de New York existe una sala de baile en la que los hombres se ponen en una fila y las mujeres en otra de tal forma que salen a bailar por parejas en el orden en el que están en la fila. Por supuesto ni un hombre ni una mujer pueden salir a bailar sólos ni quedarse en la pista sólos. Sin embargo no tienen por qué salir con la pareja con la que entraron.

#### <span style="color:cyan;">Problema</span>
Lanza distintos bailarines en el orden en el que desees y mira qué sucede. Cambia también el tiempo de baile en los distintos procesos. Por ejemplo, que a las mujeres les guste mucho más que a los hombres el quedarse bailando. ¿Qué sucede?

## <span style="color:orange;">Problema de los babuinos</span>
### <span style="color:yellow;">Enunciado del problema de los babuinos</span>
En el parque nacional Kruger en Sudafrica hay un cañón muy profundo con una simple cuerda para cruzarlo. Los babuinos necesitan cruzar ese cañón constantemente en ambas direcciones gracias a una cuerda. Sin embargo:
- Como los babuinos son muy agresivos, si dos de ellos se encuentran en cualquier punto de la cuerda yendo en direcciones opuestas, estos se pelearán y terminarán cayendo por el cañón y muriendo.
- La cuerda no es muy resistente y aguanta a un máximo de cinco babuinos simultáneamente. Si en cualquier instante hay más de cinco babuinos en la cuerda, esta se romperá y los babuinos caerán también al vacío.

#### <span style="color:cyan;">Definiendo el LightSwitch</span>
Dado que en python no es posible pasar por valor es necesario para poder mantener variables dentro del Light Switch en definirlos en forma de clase clase. En las siguientes líneas se define esa clase con dos métodos para encender y apagar la luz.

En las siguientes líneas vamos a crear los procedimientos para ser lanzados como hilos. Se puede comprobar que coinciden perfectamente con el pseudocódigo presenado en las clases de teoría.

Ya sólo nos falta probarlo. Para ello creamos hilos con babuinos que quieren ir o venir aleatoriamente. Creamos 20 de esos hilos.

Como se puede ver, esta solución no previene inanición.

#### <span style="color:cyan;">Cuestiones y problemas</span>
1. Comprueba a partir de la salida que se ha resuelto correctamente el problema.
2. Intenta modificar el resultado para que no haya inanición.

## <span style="color:orange;">Problema de la cena de los filósofos</span>
El problema de la cena de los filósofos (dining philosophers problem) es un problema clásico de las ciencias de la computación propuesto por Edsger Dijkstra en 1965 para representar el problema de la sincronización de procesos en un sistema operativo. Cabe aclarar que la interpretación está basada en pensadores chinos, quienes comían con dos palillos, donde es más lógico que se necesite el del comensal que se siente al lado para poder comer.

### <span style="color:yellow;">Enunciado del problema</span>

Cinco filósofos se sientan alrededor de una mesa y pasan su vida cenando y pensando. Cada filósofo tiene un plato de fideos y un tenedor a la izquierda de su plato. Para comer los fideos son necesarios dos tenedores y cada filósofo sólo puede tomar los que están a su izquierda y derecha. Si cualquier filósofo toma un tenedor y el otro está ocupado, se quedará esperando, con el tenedor en la mano, hasta que pueda tomar el otro tenedor, para luego empezar a comer.

Si dos filósofos adyacentes intentan tomar el mismo tenedor a una vez, se produce una condición de carrera: ambos compiten por tomar el mismo tenedor, y uno de ellos se queda sin comer.

Si todos los filósofos toman el tenedor que está a su derecha al mismo tiempo, entonces todos se quedarán esperando eternamente, porque alguien debe liberar el tenedor que les falta. Nadie lo hará porque todos se encuentran en la misma situación (esperando que alguno deje sus tenedores). Entonces los filósofos se morirán de hambre. Este bloqueo mutuo se denomina interbloqueo o _deadlock_.

El problema consiste en encontrar un algoritmo que permita que los filósofos no se mueran de hambre.

### <span style="color:yellow;">Solución</span>
A continuación se presenta una solución para el problema en el que no se previene inanición de algún filósofo.

#### <span style="color:cyan;">Cuestiones</span>
1. Identifica qué hacen los procedimientos __izq__ y __der__
2. El procedimiento más importante es el procedimiento __intenta__. Identifica su funcionamiento e identifica por qué cuando un filósofo deja los tenedores hace que lo intenten los filósofos que están a su izquierda y a su derecha.

## <span style="color:orange;">Otros problemas</span>
En las siguientes celdas se van a enunciar un conjunto de problemas típicos para que pruebes a solucionarlos con las herramientas que ya dispones. Todos ellos han sido extraidos del libro ["El pequeño libro de los semáforos"](https://greenteapress.com/wp/semaphores/).

Para aprender bien se aconseja que añadas una o varias celdas después de cada problema y que sigas el siguiente procedimiento para resolverlos.
1. Analiza el problema en papel.
    1. Identifica los procesos que deben existir.
    2. Identifica los puntos de concurrencia y ordénalos en cada proceso.
    3. Encuentra una forma de solucionar cada punto de concurrencia.
    4. Añade los _mutex_ necesarios para proteger la memoria compartida para que no se produzcan condiciones de carrera.
    5. Analiza el código resultante y comprueba que no hay problemas de interbloqueos.
    6. Analiza el problema y comprueba si hay problemas de inanición y si estos son resolubles sin desaprovechar recursos.
2. Impleméntalo usando los recursos disponibles en este "_notebook_".
3. Añade las salidas necesarias para comprobar que funciona correctamente.
4. Si algo no funciona vuelve al punto 1.

### <span style="color:yellow;">Problema de los lectores-escritores</span>
Este problema se refiere a cualquier situación en la que una estructura de datos, base de datos o sistema de archivos es leído y modificado por hilos concurrentes. Mientras se está escribiendo o modificando la estructura de datos, a menudo es necesario impedir la lectura de otros hilos, para evitar que un lector interrumpa una modificación en curso y lea de forma inconsistente, o bien datos no válidos. Como en el problema productor-consumidor, la solución es asimétrica. Los lectores y escritores ejecutan códigos diferentes antes de entrar en la sección crítica. Las restricciones de sincronización son:
1. Cualquier número de lectores puede estar en la sección crítica simultáneamente.
2. Los escritores deben tener acceso exclusivo a la sección crítica. En otras palabras, un escritor no puede entrar en la sección crítica mientras que cualquier otro hilo (lector o escritor) está allí, y mientras el escritor está allí, ningún otro hilo puede entrar.

### <span style="color:yellow;">Problema de los lectores-escritores con prioridad a escritores</span>
Es el mismo problema que el anterior pero si un escritor desea escribir no puede leer ningún lector nuevo hasta que no se realice la o las escrituras.

### <span style="color:yellow;">Problema de la cena de los salvajes</span>
Una tribu de salvajes realiza cenas comunales de una olla grande que puede contener M porciones de misioneros guisados. Cuando un salvaje quiere comer, se sirve de la olla, a menos que esté vacía. Si la olla está vacía, el salvaje despierta al cocinero que cocina a un nuevo misionero y luego espera hasta que el cocinero haya vuelto a llenar la olla. Evidentemente si otro quiere cenar, la olla está vacía pero el cocinero está despierto no tiene que despertarlo pero esperará a que llene la olla.

**Variante**: Se pueden concinar sólo hasta N misioneros que fueron el total de misioneros capturados en la última cacería de misioneros.

### <span style="color:yellow;">Problema del H<sub>2</sub>O</span>
Este problema ha sido un elemento básico de la clase de sistemas operativos en la Universidad de California en Berkeley durante al menos una década. 

Hay dos tipos de átomos: oxígeno e hidrógeno, y queremos crear moléculas de agua. Para ello tenemos un catalizador que espera a que estén disponibles dos átomos de hidrógeno y uno de oxígeno para porducir una molécula de agua.
Para ensamblar estos átomos en moléculas de agua tenemos que crear una barrera que haga que cada átomo espere a que estén disponibles el resto  espere hasta que una molécula completa esté lista para proceder. A medida que cada hilo pasa la barrera, debe invocar la unión. Debes garantizar que todos los hilos de una molécula invoquen la unión antes de que lo haga cualquiera de los hilos de la siguiente molécula. Como los átomos son indistinguibles basta con que haya dos hidrógenos y un oxígeno para que puedan reaccionar.

### <span style="color:yellow;">Problema de la montaña rusa</span>
Supongamos que hay n de pasajeros y un coche. Los pasajeros esperan repetidamente para dar una vuelta en la montaña rusa, que puede contener C pasajeros. El coche de la montaña rusa espera a estar lleno para empezar el viaje.

Aquí hay algunos detalles adicionales:
- Los pasajeros deben invocar el _embarque_ y _desembarque_.
- El coche debe invocar la _carga_, _corre_ y _descarga_.
- Los pasajeros no pueden embarcar hasta que el coche haya invocado la carga.
- El coche no puede salir hasta que se haya llenado.
- Los pasajeros no pueden desembarcar hasta que el coche no haya invocado la descarga.

### <span style="color:yellow;">Problema del bar de sushi</span>
Imagina un bar de sushi con 5 asientos. Si llega un cliente y hay un asiento vacío, puede tomar asiento inmediatamente. Pero si cuando llega cuando los 5 asientos están llenos, eso significa que pueden están cenando juntos, y tendrá que esperar a que todos se vayan antes de sentarse.

### <span style="color:yellow;">La tarta de cumpleaños</span>

En un cumpleaños de ninos pequeños se pretende repartir trozos de tarta a todos los niños. Como un día
especial que es, los niños pueden repetir tantas veces como quieran y cuando quieran pero usando un sistema de rondas, de tal forma que hasta que todos los niños no han recibido su trozo de tarta de una ronda, los niños que ya han tomado tarta no pueden repetir.

Implementa el proceso **ninoPideTarta(int NumNino)** teniendo en cuenta que los semáforos que se van a utilizar son semáforos blandos (tambien denominados debiles). Se dispone de la funcion auxiliar **comeTarta()** para representar la tarea de comer el trozo de tarta. De igual forma, se utilizarán las funciones **up(semaforo)** y **down(semaforo)** para manipular los semáforos. Se pueden crear otras funciones si se consideran necesarias.

### <span style="color:yellow;">Problema de Santa Klaus</span>

Santa Klaus es un hombre muy mayor y necesita dormir mucho; por ello aprovecha siempre que puede para
dormir.

Los elfos saben de su necesidad y han decidido que aunque haya un problema en la fábrica de juguetes no
le despertarán, pero cuando haya tres problemas entonces sí que lo haran.

La fábrica y la oficina de Santa estan muy alejadas, por lo que si un elfo va en busca de Santa no vuelve hasta que Santa resuelva su problema.

Los elfos irán a despertar a Santa pero si no hay otros dos elfos esperando es que no hay suficientes problemas como para despertar a Santa, por ello y ya que no tienen nada que hacer se quedarán a la espera. El tercer elfo hará lo mismo pero despertara a Santa. Cuando Santa resuelva los problemas en la fábrica los tres elfos regresarán a la fábrica.

Así mismo los renos estan de vacaciones todo el año y vuelven por navidad para tirar del trineo de Santa. Cuando llegue el ultimo de los 9 renos despertará a Santa para irse a repartir los juguetes.

No es necesario controlar lo que sucede con los elfos y la produccion cuando Santa esta de reparto.

### <span style="color:yellow;">El baño unisex de la EHS</span>

En la Escuela de Hacking Superior todos los aseos son unisex. Para usar estos aseos hay un selector a la entrada que tiene tres posiciones: hombres, mujeres y libre. El cartel de hombres lo pone el primer hombre que entre, el de mujeres la primera mujer que entre y el de libre lo pone cualquiera al salir siempre que quede vacío el aseo. Cuando esta puesto el cartel de mujeres sólo pueden entrar mujeres y cuando pone el cartel de hombres sólo pueden entrar hombres. En el aseo sólo caben 10 personas por lo que cuando está lleno se activara un cartel que indica que está lleno y los que lleguen esperarán fuera a que haya sitio.

Resuelve  el  problema  implementando  el  pseudocodigo  de  los  procesos **mujer()** y de **hombre()** para que cumplan las condiciones. No es necesario prevenir la inanición.

### <span style="color:yellow;">Cruce automático</span>

En el ano 2056 se realiza una modificación en el código de la circulación en la que cambia la regulación de los cruces con bajo tráfico. Dada la capacidad de las comunicaciones inalambricas cuando un coche se acerca a un cruce  reserva los elementos necesarios del cruce para poder realizar la maniobra.

Si no puede acceder a una de las zonas necesarias para realizar la maniobra no accederá al cruce. Es muy importante que si las maniobras de varios coches no interfieren todos los coches puedan acceder al cruce sin ningún problema.

Crea el proceso **coche(origen,destino)** donde origen y destino determinan las zonas a utilizar. En el esquema adjunto se puede ver la numeración de zonas así como la numeración de origen/destino. Por ejemplo, si el origen es 2 y el destino es 1 es necesario reservar las zonas Z3, Z4, Z1, sin embargo de 1 a 2 hace falta reservar solo Z2. 

Utiliza las primitivas **up(sem)**, **down(sem)**, **up(sem,n)** y **down(sem,n)** (las dos últimas para hacer ups y downs multiples si es necesario). No es necesario prevenir inanición pero sí evitar interbloqueos.

### <span style="color:yellow;">Cadena de montaje</span>
En una fábrica de coches hay 5 cadenas de montaje. De cada una de ellas salen los coches de uno en uno. Estos coches son almacenados en una zona de espera en la que caben 10 coches. En esta zona de espera hay 3 dársenas donde paran los camiones de transporte que pueden transportar cada uno 6 coches. Los camiones nunca parten antes de estar completamente cargados. Cuando un camion desea cargar puede hacerlo siempre y cuando uno que estuviese cargando pueda terminar la carga.

Implementa los procesos **cadenaMontaje** y **cargaCamion**.

**No es necesario** controlar las posiciones donde se encuentran los coches a cargar ni implementar el acceso a las darsenas ni prevenir inanición.

### <span style="color:yellow;">Visita a la Sagrada Familia y museo</span>

La basílica de la Sagrada Familia en Barcelona dispone de un museo en el que se explican los métodos de construcción y maquetas de prueba de carga. los responsables de la visita de turistas a la basílica están hartos de que los turistas entren a ver la basílica sin saber lo que están viendo y han decidido que es necesario visitar el museo antes de entrar en la basílica. Además, han decidido que la basílica solo se pueda visitar con un guía que explique lo que se está viendo. Nos piden implementar un sistema que permita realizar esto de forma eficiente y con las siguientes condiciones:
* El número maximo de personas en el museo es de 100 personas.
* Cuando un visitante abandona el museo se pone en la cola de espera para que los recoja un guía.
* Hay 5 guías.
* Un guía recoge a grupos de 10 personas.
* Si no hay 10 personas en la cola de espera el guía espera a que las haya y si hay más solo coge a 10

### <span style="color:yellow;">El aeropuerto de Umea</span>

Umea es una pequeña ciudad sueca en el círculo polar ártico. Debido a las inclemencias climáticas su principal medio de transporte es un pequeño avión con un máximo de 32 plazas para viajeros. El pequeño aeropuerto tiene una única terminal de embarque. Con el fin de obtener rentabilidad en la línea de comunicación con Estocolmo y reducir costes, la línea aérea se usa de forma muy similar a la de una línea de autobuses pero por seguridad con unas pocas reglas que es necesario cumplir:
1.  Cuando el avión está preparado, éste se llena con los pasajeros que estén esperando. Si hay más de 32 pasajeros esperando se llenará con 32 sin un orden determinado. Los pasajeros que no hayan podido embarcar esperarán al siguiente avión.
2.  Si no hubiera pasajeros esperando, el avión volverá de forma inmediata a Estocolmo.
3.  Si los pasajeros están embarcando nadie que llegue después de que se inicie el embarque podrá entrar en la zona de embarque y mucho menos embarcar (independientemente de si los pasajeros llenan el avión o no) y sólo podrán entrar en la zona de embarque tras la partida del avión.

### <span style="color:yellow;">Los pasos de cebra de Navalaoveja</span>

En el ayuntamiento de Navalaoveja se han gastado todo el presupuesto en mármol. Como no tienen presupuesto suficiente han despedido a policías municipales pero aún así tienen que mantener la seguridad en los pasos de cebra a la salida de los colegios. Como no hay presupuesto suficiente nos piden que implementemos un sistema basado en semáforos para que tanto los peatones como los automóviles cumplan con las condiciones siguientes sin necesidad de policías:


* Si no hay automóvil que quieran acceder al paso de cebra los peatones deben poder cruzar cuando lo deseen.
* Si no hay peatones que quieran acceder al paso de cebra los automóviles deben poder pasar cuando lo deseen.
* Si hay diez automóviles esperando a que pasen peatones deben dejar de cruzar peatones y pasar los 10 coches. Aunque haya más coches, sólo pasarán 10 si hay peatones esperando.
* Salvo en el caso anterior los peatones siempre tienen prioridad para cruzar.
* Como el uso del paso de cebra en ambas direcciones tanto de peatones como de automóviles no es exclusivo, no se tendrá en cuenta la dirección en la que desean pasar.
* Evidentemente los peatones pueden pasar en grupos pero los coches sólo pueden pasar de uno en uno. 

    Implementa los procesos **peaton** y **coche()** usando semáforos. Para ello puedes usar las funciones up(sem), down(sem). También puedes usar las funciones llegapeaton(), llegacoche(), cruzapeaton(), cruzacoche(), continuapeaton() y continuacoche().